In [1]:
import pandas as pd

In [2]:
df_baseSNII = pd.read_csv('datasetF.csv')
df_baseSNII

In [3]:
df_basePATENTE = pd.read_csv('InventoresPatentes.csv')
df_basePATENTE

In [4]:
df_lista = pd.read_csv('Nombres_PxS.csv')
df_lista

In [5]:
df_idPatentes = pd.read_csv('id_Patentes.csv')
df_idPatentes

---

In [6]:
# Vamos a aliminar la columna 'a' del df_idPatentes
df_idPatentes.drop('a', axis=1, inplace=True)

# Normalizamos los acentos,puntos, guion, y las letras que tengan diéresis como ö,en el 'df_idPatentes' en la columna 'Inventor'
df_idPatentes['Inventor'] = df_idPatentes['Inventor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df_idPatentes

In [7]:
# Vamos a hacer que se busque el nombre del inventor en la lista de nombres de la columna 'Inventor' del 'df_idPatentes' en la columna 'Inventor' del 'df_basePATENTE'
# si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_basePATENTE' llamada 'inventor_id' con el id del inventor que se encontró en la lista
df_basePATENTE['inventor_id'] = df_basePATENTE['Inventor'].apply(lambda x: df_idPatentes[df_idPatentes['Inventor'].str.contains(x, case=False, na=False)]['inventor_id'].values[0] if len(df_idPatentes[df_idPatentes['Inventor'].str.contains(x, case=False, na=False)]) > 0 else None)

df_basePATENTE

In [10]:
# Vamos a buscar en el 'df_lista' los nombres de los inventores que se encuentran en la columna 'NOMBRE_INVENTOR' del 
# 'df_basePATENTE' y si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_lista' 
# llamada 'inventor_id' con el id del inventor que se encontró en 'df_basePATENTE'
df_lista['inventor_id'] = df_lista['NOMBRE_INVENTOR'].apply(lambda x: df_basePATENTE[df_basePATENTE['NOMBRE_INVENTOR'].str.contains(x, case=False, na=False)]['inventor_id'].values[0] if len(df_basePATENTE[df_basePATENTE['NOMBRE_INVENTOR'].str.contains(x, case=False, na=False)]) > 0 else None)

df_lista

In [11]:
# Vamos a buscar en el 'df_lista' los nombres de los inventores que se encuentran en la columna 'INVESTIGADOR' 
# del 'df_baseSNII' y si se encuentra el nombre en la lista, se va a agregar una nueva columna en el 'df_lista' 
# llamada 'CVU' con el id del inventor que se encontró en 'df_baseSNII'
df_lista['CVU'] = df_lista['NOMBRE DEL INVESTIGADOR'].apply(lambda x: df_baseSNII[df_baseSNII['INVESTIGADOR'].str.contains(x, case=False, na=False)]['CVU'].values[0] if len(df_baseSNII[df_baseSNII['INVESTIGADOR'].str.contains(x, case=False, na=False)]) > 0 else None)

df_lista

In [ ]:
# guardamos el resultado en un archivo csv
df_lista.to_csv('Nombres_PxS.csv', index=False)